# To predict whether a person survived or not in Titanic using titanic dataset# To predict whether a person survived or not in Titanic using titanic dataset

### Initialize and create a spark session

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("Titanic").getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://Varun-CK:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1577562367599)
SparkSession available as 'spark'


2019-12-29 01:16:22 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3d5a1ad0


### Initialize Logger

In [2]:
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j._


### Import statements to setup ML for Logistic Regression

In [3]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{StringIndexer,VectorAssembler,OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{StringIndexer, VectorAssembler, OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors


### Using Spark to read the titanic data set

In [4]:
val data = spark.read.options(Map(("header","true"),("inferSchema","true"))).csv("titanic.csv")

data: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 10 more fields]


### Printing the first row of the dataframe

In [7]:
data.head(1)

res2: Array[org.apache.spark.sql.Row] = Array([1,0,3,Braund, Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.25,null,S])


### Printing the schema of the dataframe

In [8]:
data.printSchema

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### Show

In [9]:
data.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

### Checking out whether the string columns "Sex", "Name", "Ticket", "Cabin" and "Embarked" are useful or not

In [10]:
data.groupBy("Sex").count().count()

res5: Long = 2


In [11]:
data.groupBy("Name").count().count()

res6: Long = 891


In [12]:
data.groupBy("Ticket").count().count()

res7: Long = 681


In [13]:
data.groupBy("Cabin").count().count()

res8: Long = 148


In [14]:
data.groupBy("Embarked").count().count()

res9: Long = 4


In [15]:
data.count()

res10: Long = 891


### Checking whether the feature "PassengerId" is useful or not

In [16]:
data.groupBy("PassengerId").count().count()

res11: Long = 891


### Filtering the data and dropping columns that are not useful

In [17]:
var filtered_data = data.drop("Name","Ticket","Cabin","PassengerId")

filtered_data: org.apache.spark.sql.DataFrame = [Survived: int, Pclass: int ... 6 more fields]


### Count

In [18]:
filtered_data.count()

res12: Long = 891


### Count by dropping duplicates

In [19]:
filtered_data.na.drop().count()

res13: Long = 712


In [20]:
filtered_data = filtered_data.na.drop()

filtered_data: org.apache.spark.sql.DataFrame = [Survived: int, Pclass: int ... 6 more fields]


### Since the "Sex" and "Embarked" columns are categorical type, we can convert it to numerical and to vector using StringIndexer and OneHotEncoder

#### Using String Indexer to convert categorical string columns to numerical type

In [21]:
val genderIndexer = new StringIndexer().setInputCol("Sex").setOutputCol("SexInd")
val embarkedIndexer = new StringIndexer().setInputCol("Embarked").setOutputCol("EmbarkedInd")

genderIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_d47025f0974a
embarkedIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_d6c867db5410


#### Using One Hot Encoder to convert categorical numeric type columns to Vector type

In [22]:
val genderEncoder = new OneHotEncoder().setInputCol("SexInd").setOutputCol("SexVec")
val embarkedEncoder = new OneHotEncoder().setInputCol("EmbarkedInd").setOutputCol("EmbarkedVec")

genderEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_101c9ae34dc6
embarkedEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_d818e7b28330


### Assembling all the features to a single vector column "features"

In [23]:
filtered_data.printSchema

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [24]:
val assembler = new VectorAssembler().setInputCols(Array("Pclass","SexVec","Age","SibSp","Parch","Fare","EmbarkedVec"))
                                     .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_fa616292faee


### Splitting the resultant data into training data and testing data,

<code>
<b>Training data is to train the model</b>
<b>Testing data is to test the builted model</b>
</code>

In [25]:
val Array(train_data,test_data) = filtered_data.randomSplit(Array(0.7,0.3))

train_data: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Survived: int, Pclass: int ... 6 more fields]
test_data: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Survived: int, Pclass: int ... 6 more fields]


In [26]:
filtered_data.describe().show()

+-------+------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+
|summary|          Survived|            Pclass|   Sex|              Age|             SibSp|              Parch|              Fare|Embarked|
+-------+------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+
|  count|               712|               712|   712|              712|               712|                712|               712|     712|
|   mean|0.4044943820224719| 2.240168539325843|  null|29.64209269662921|0.5140449438202247|0.43258426966292135| 34.56725140449432|    null|
| stddev|0.4911389472541192|0.8368543166903446|  null|14.49293290032352|0.9306921267673427| 0.8541814457454133|52.938648174710906|    null|
|    min|                 0|                 1|female|             0.42|                 0|                  0|               0.0|       C|
|    max|           

In [27]:
train_data.describe().show()

+-------+------------------+------------------+------+------------------+------------------+------------------+------------------+--------+
|summary|          Survived|            Pclass|   Sex|               Age|             SibSp|             Parch|              Fare|Embarked|
+-------+------------------+------------------+------+------------------+------------------+------------------+------------------+--------+
|  count|               500|               500|   500|               500|               500|               500|               500|     500|
|   mean|             0.406|             2.236|  null|          29.53666|             0.506|              0.46| 35.39161619999997|    null|
| stddev|0.4915763364821157|0.8448262915687766|  null|14.164700028528545|0.9379357370810582|0.8933062140072503|50.288984425336935|    null|
|    min|                 0|                 1|female|              0.42|                 0|                 0|               0.0|       C|
|    max|           

In [28]:
test_data.describe().show()

+-------+-------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+
|summary|           Survived|            Pclass|   Sex|              Age|             SibSp|              Parch|              Fare|Embarked|
+-------+-------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+
|  count|                212|               212|   212|              212|               212|                212|               212|     212|
|   mean| 0.4009433962264151|              2.25|  null|29.89075471698113|0.5330188679245284|0.36792452830188677| 32.62299481132077|    null|
| stddev|0.49124954962175477|0.8196346429155487|  null|15.27080628343462|0.9152962070071883| 0.7519386848315638|58.798179287125635|    null|
|    min|                  0|                 1|female|             0.67|                 0|                  0|               0.0|       C|
|    max|    

### Creating a logistic regression model object

In [29]:
val lor = new LogisticRegression().setLabelCol("Survived").setFeaturesCol("features")

lor: org.apache.spark.ml.classification.LogisticRegression = logreg_404173553396


### Setting Up the Pipeline

In [30]:
import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.Pipeline


In [31]:
val pipeline = new Pipeline().setStages(Array(genderIndexer,embarkedIndexer,genderEncoder,embarkedEncoder,assembler,lor))

pipeline: org.apache.spark.ml.Pipeline = pipeline_2a25a01ffe07


### Fitting the pipeline to training set

In [32]:
val pipelineModel = pipeline.fit(train_data)

2019-12-29 01:34:59 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-12-29 01:34:59 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_2a25a01ffe07


### Getting Results on Test Set

In [33]:
val results = pipelineModel.transform(test_data)

results: org.apache.spark.sql.DataFrame = [Survived: int, Pclass: int ... 14 more fields]


In [34]:
results.printSchema

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexInd: double (nullable = false)
 |-- EmbarkedInd: double (nullable = false)
 |-- SexVec: vector (nullable = true)
 |-- EmbarkedVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [35]:
val output = results.select("Survived","rawPrediction","prediction","probability","features")

output: org.apache.spark.sql.DataFrame = [Survived: int, rawPrediction: vector ... 3 more fields]


In [36]:
output.show(5)

+--------+--------------------+----------+--------------------+--------------------+
|Survived|       rawPrediction|prediction|         probability|            features|
+--------+--------------------+----------+--------------------+--------------------+
|       0|[-0.8167804853568...|       1.0|[0.30644750053728...|[1.0,1.0,18.0,1.0...|
|       0|[-0.2056929238039...|       1.0|[0.44875731315026...|[1.0,1.0,19.0,1.0...|
|       0|[-1.0404908334359...|       1.0|[0.26105529837675...|[1.0,1.0,22.0,0.0...|
|       0|[-0.8269364091300...|       1.0|[0.30429323975603...|[1.0,1.0,30.0,0.0...|
|       0|[-0.5223945903319...|       1.0|[0.37229246907609...|[1.0,1.0,36.0,0.0...|
+--------+--------------------+----------+--------------------+--------------------+
only showing top 5 rows



### MODEL EVALUATION

#### 1) Converting the data to rdd and evaluating using MulticlassMetrics to print the confusion matrix

In [37]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics

import org.apache.spark.mllib.evaluation.MulticlassMetrics


In [38]:
val clean_result = output.withColumn("Survived",output("Survived").cast("double"))

clean_result: org.apache.spark.sql.DataFrame = [Survived: double, rawPrediction: vector ... 3 more fields]


In [39]:
clean_result.select("Survived","prediction").show(5)

+--------+----------+
|Survived|prediction|
+--------+----------+
|     0.0|       1.0|
|     0.0|       1.0|
|     0.0|       1.0|
|     0.0|       1.0|
|     0.0|       1.0|
+--------+----------+
only showing top 5 rows



In [40]:
val predictionAndLabel = clean_result.select("Survived","prediction").as[(Double,Double)].rdd

predictionAndLabel: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[185] at rdd at <console>:56


In [41]:
val metrics = new MulticlassMetrics(predictionAndLabel)

metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@6bac10d8


##### Printing the confusion matrix

In [42]:
println(metrics.confusionMatrix)

110.0  28.0  
17.0   57.0  


##### Printing the Accuracy

In [43]:
println(metrics.accuracy)

0.7877358490566038


##### Recall

In [44]:
println(metrics.recall)

0.7877358490566038


##### precision

In [45]:
println(metrics.precision)

0.7877358490566038


#### 2) Evaluating using BinaryClassificationEvaluator

In [46]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator


In [47]:
val bin_eval = new BinaryClassificationEvaluator().setRawPredictionCol("rawPrediction").setLabelCol("Survived")

bin_eval: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_715c0512f293


##### Calculating Area Under ROC

In [48]:
val AOC =bin_eval.evaluate(output)

AOC: Double = 0.8708198239925896


#### 3) Evaluating using MulticlassClassificationEvaluator

In [49]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


In [50]:
val multi_eval = new MulticlassClassificationEvaluator().setPredictionCol("prediction").setLabelCol("Survived")

multi_eval: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_fed23b672fb6


##### Calculating Area Under ROC

In [51]:
val AOC_2 = multi_eval.evaluate(output)

AOC_2: Double = 0.7847988608045567


##### Printing Area Under ROC

In [52]:
println(AOC_2)

0.7847988608045567


### Stopping the created spark session

In [53]:
spark.stop()

## Thank You!